In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [2]:
import json

# upload the files to the colab environment
file_path = '/content/intents.json'

with open(file_path, 'r') as file:
    data = json.load(file)




In [3]:

# Load and prepare the dataset


training_sentences = []
training_labels = []
labels = []
responses = {}
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses[intent['tag']] = intent['responses']
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Tokenization and Padding
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, padding='post')

# Label Encoding
label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
training_labels_encoded = label_encoder.transform(training_labels)

# Model Definition
model = Sequential([
    Embedding(2000, 16, input_length=len(padded_sequences[0])),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(len(labels), activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, training_labels_encoded, epochs=150, verbose=1)

# Sentiment Analysis
sia = SentimentIntensityAnalyzer()

import json

# Save context data to a file in Google Colab's environment
def save_context(user_id, context_data):
    # Ensure directory exists
    import os
    os.makedirs('/content/user_data', exist_ok=True)  # Create the directory if it does not exist
    file_path = f'/content/user_data/{user_id}_context.json'  # Set the file path
    with open(file_path, "w") as file:
        json.dump(context_data, file, indent=4)  # Save the context data to a JSON file

# Load context data from a file in Google Colab's environment
def load_context(user_id):
    file_path = f'/content/user_data/{user_id}_context.json'  # Set the file path
    try:
        with open(file_path, "r") as file:
            return json.load(file)  # Load the context data from a JSON file
    except FileNotFoundError:
        return [] # Return an empty list if the file does not exist




# Generate Response
def generate_response(message, user_id):
    sentiment = sia.polarity_scores(message)['compound']
    sequences = tokenizer.texts_to_sequences([message])
    padded = pad_sequences(sequences, maxlen=len(padded_sequences[0]), padding='post')
    predictions = model.predict(padded)
    tag = label_encoder.inverse_transform([np.argmax(predictions)])[0]

    # Load previous context
    previous_context = load_context(user_id)

    # Adjust responses based on sentiment
    response = np.random.choice(responses[tag])
    if sentiment < -0.5:
        response = "It seems you're feeling quite down. " + response
    elif sentiment > 0.5:
        response = "I'm glad to see you're feeling well! " + response

    # Update and save the current context
    previous_context.append(tag)
    save_context(user_id, previous_context)

    return response

# Chat Function
def chat():
    user_id = input("Please enter your user ID to start: ")

    if load_context(user_id):
        print(f"Welcome back ,{user_id}!.How are u feeling now?Type 'quit' to exit ")
    else:
        print(f"Hello,{user_id}! I'm Pandora, your Personal Therapeutic AI Assistant. How can I assist you today?.Type 'quit' to exit!")


    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print("Goodbye! Feel free to return if you need to talk.")
            break
        response = generate_response(inp, user_id)
        print("Pandora:", response)




Epoch 1/150
8/8 [==============================] - 5s 17ms/step - loss: 4.3797 - accuracy: 0.0431
Epoch 2/150
8/8 [==============================] - 0s 21ms/step - loss: 4.3702 - accuracy: 0.0517
Epoch 3/150
8/8 [==============================] - 0s 26ms/step - loss: 4.3565 - accuracy: 0.0517
Epoch 4/150
8/8 [==============================] - 0s 27ms/step - loss: 4.3355 - accuracy: 0.0517
Epoch 5/150
8/8 [==============================] - 0s 27ms/step - loss: 4.2882 - accuracy: 0.0517
Epoch 6/150
8/8 [==============================] - 0s 25ms/step - loss: 4.1997 - accuracy: 0.0517
Epoch 7/150
8/8 [==============================] - 0s 27ms/step - loss: 4.1405 - accuracy: 0.0517
Epoch 8/150
8/8 [==============================] - 0s 25ms/step - loss: 4.0750 - accuracy: 0.0517
Epoch 9/150
8/8 [==============================] - 0s 27ms/step - loss: 4.0034 - accuracy: 0.0517
Epoch 10/150
8/8 [==============================] - 0s 28ms/step - loss: 3.9346 - accuracy: 0.0517
Epoch 11/150
8/8 [=

In [4]:
#Chat with the AI assistant.
chat()

Please enter your user ID to start: harsha
Hello,harsha! I'm Pandora, your Personal Therapeutic AI Assistant. How can I assist you today?.Type 'quit' to exit!
You: quit
Goodbye! Feel free to return if you need to talk.
